In [20]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

In [21]:
tf.enable_eager_execution()

In [37]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "arch_weight_decay": 1e-3,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate_min": 0.001,
    "learning_rate": 0.025,
    "unrolled": True,
    "epochs": 10,
    "batch_size": 4,
    "save": "EXP",
    
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [4, 4],
    "save_checkpoints_steps": 100,
    "model_dir": './outputdir'
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [38]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn():
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = 16, 16

        # Create a dictionary describing the features.  
        image_feature_description = {
            'name': tf.FixedLenFeature([], tf.string),  
            'label_encoded': tf.FixedLenFeature([], tf.string),
            'encoded': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            image= feature['encoded']
            label = feature['label_encoded']
            name = feature['name']

            image = tf.image.decode_png(image, channels=3)
            label = tf.image.decode_png(label, channels=3)

            image = tf.cast(image, tf.float32)
            image = tf.image.resize(image, (W, H))
            label = tf.cast(label, tf.float32)
            label = tf.image.resize(label, (W, H))

            return image, label

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)

        return dataset
    
    return _input_fn

In [39]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn():
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
            x_train = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, 6, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            x_valid = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_valid = np.random.randint(0, 6, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            
            ds = (x_train, x_valid), (y_train, y_valid)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, 6, (NUM_IMAGES, W, H, 1)).astype(np.float32)
            
            ds = (x_train, y_train)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = 1 # end-of-input after this
#         w, h, c = dataset.shape
        dataset = dataset.repeat(num_epochs)
        return dataset
    
    return _input_fn

### Model function

In [53]:
def model_fn(features, labels, mode):
    criterion = tf.losses.sigmoid_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion)
    optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)
    criterion = tf.losses.sigmoid_cross_entropy
    
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
        # architect step
        architect_step = architect.step(input_train=x_train,
                                        target_train=y_train,
                                        input_valid=x_valid,
                                        target_valid=y_valid,
                                        unrolled=args.unrolled,
                                        )


        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)
        grads = tf.gradients(loss, w_var)
        clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
        train_op = optimizer.apply_gradients(zip(clipped_gradients, w_var), global_step=tf.train.get_global_step())

        miou = tf.metrics.mean_iou(
                    labels=y_train,
                    predictions=preds,
                    num_classes=args.num_classes
                )
        acc = tf.metrics.accuracy(labels=y_train, 
                                  predictions=preds)
        eval_metric_ops = {
            "miou": miou,
            "accuracy": acc
        }
        
    elif mode == tf.estimator.ModeKeys.EVAL:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        init = tf.initialize_all_variables()
        with tf.control_dependencies([init]):
            preds = model(x_valid)
            loss = model._loss(preds, y_valid)
            miou = tf.metrics.mean_iou(
                        labels=y_valid,
                        predictions=preds,
                        num_classes=args.num_classes
                    )
            acc = tf.metrics.accuracy(labels=y_valid, predictions=preds)
            eval_metric_ops = {
                "miou": miou,
                "accuracy": acc
            }
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.EstimatorSpec(
      mode = mode,
      predictions = prediction_dict,
      loss = loss,
      train_op = train_op,
      eval_metric_ops = eval_metric_ops,
      export_outputs = export_outputs)

### Estimator

In [54]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = '../../datasets/infer/infer-00000-00007.tfrecords',
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.batch_size)

In [55]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


In [56]:
config = tf.estimator.RunConfig(save_checkpoints_steps=args.save_checkpoints_steps,
                                model_dir=args.model_dir)

In [57]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir):
    estimator = tf.estimator.Estimator(model_fn = model_fn, 
                         config=config)
    train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                    max_steps = 1000)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_train(),
                                  steps = None, throttle_secs=600)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [ ]:
train_and_evaluate(args.model_dir)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0f7eccbf50>, '_model_dir': './outputdir', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation l